# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame

output_data = "../output_data/cities.csv"

city_data_df = pd.read_csv(output_data, index_col="City_ID")


# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,tofol,5.3248,163.0078,28.27,75,100,3.65,FM,1697299987
1,margaret river,-33.9500,115.0667,13.41,81,40,6.13,AU,1697299987
2,edinburgh of the seven seas,-37.0676,-12.3116,14.72,96,100,10.38,SH,1697299987
3,bredasdorp,-34.5322,20.0403,16.41,78,83,9.67,ZA,1697299987
4,tsiombe,-25.3000,45.4833,24.60,74,27,5.25,MG,1697299988


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

narrow_citydata_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 2)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
narrow_citydata_df.dropna()

# Display sample data
narrow_citydata_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
20,siwa oasis,29.2041,25.5195,26.47,31,0,3.54,EG,1697299990
124,junagarh,19.8667,82.9333,24.63,64,0,1.37,IN,1697300012
134,los angeles,34.0522,-118.2437,23.02,80,0,0.00,US,1697299304
200,buenos aires,-34.6132,-58.3772,22.22,45,0,2.64,AR,1697299462
241,satkania,22.0796,92.0572,22.26,90,0,1.75,BD,1697300036
245,san felice circeo,41.2372,13.0942,24.45,81,0,1.97,IT,1697300036
281,comala,19.3167,-103.7500,26.76,83,0,0.00,MX,1697300044
365,zadar,44.1197,15.2422,22.34,83,0,2.57,HR,1697300062
418,sciacca,37.5082,13.0881,25.57,42,0,4.43,IT,1697300074


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = narrow_citydata_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
20,siwa oasis,EG,29.2041,25.5195,31,
124,junagarh,IN,19.8667,82.9333,64,
134,los angeles,US,34.0522,-118.2437,80,
200,buenos aires,AR,-34.6132,-58.3772,45,
241,satkania,BD,22.0796,92.0572,90,
245,san felice circeo,IT,41.2372,13.0942,81,
281,comala,MX,19.3167,-103.7500,83,
365,zadar,HR,44.1197,15.2422,83,
418,sciacca,IT,37.5082,13.0881,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
siwa oasis - nearest hotel: فندق الكيلانى
junagarh - nearest hotel: No hotel found
los angeles - nearest hotel: DoubleTree by Hilton Hotel Los Angeles Downtown
buenos aires - nearest hotel: Hotel City Baires
satkania - nearest hotel: Mayer Doya Hotel & Jhal Bitan
san felice circeo - nearest hotel: Centro Ferie Salvatore
comala - nearest hotel: Hotel Brium
zadar - nearest hotel: No hotel found
sciacca - nearest hotel: Le casette di Vita
lalmohan - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
20,siwa oasis,EG,29.2041,25.5195,31,فندق الكيلانى
124,junagarh,IN,19.8667,82.9333,64,No hotel found
134,los angeles,US,34.0522,-118.2437,80,DoubleTree by Hilton Hotel Los Angeles Downtown
200,buenos aires,AR,-34.6132,-58.3772,45,Hotel City Baires
241,satkania,BD,22.0796,92.0572,90,Mayer Doya Hotel & Jhal Bitan
245,san felice circeo,IT,41.2372,13.0942,81,Centro Ferie Salvatore
281,comala,MX,19.3167,-103.7500,83,Hotel Brium
365,zadar,HR,44.1197,15.2422,83,No hotel found
418,sciacca,IT,37.5082,13.0881,42,Le casette di Vita


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_city = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_city

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)